In [ ]:
### Important Parameters for AWS Setting

In [ ]:
#AWS Related Parameters. 
aws_profile_name='' #configure your AWS local profile name
aws_service_name='bedrock-runtime' #bedrock runtime service
aws_region_name=''  #region name
opensearch_host_name = "" #pick this value from AWS OpenSearchConsole
aws_embedding_model='amazon.titan-embed-text-v1' #embedding model used

#OpenSearch Related Parameters
index_name='sgrealestate-index'

### Import Python modules

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, helpers
import boto3
from botocore.config import Config
import json

### Create AWS Opensearch Client using Boto3

In [ ]:
boto_config = Config(
    region_name = aws_region_name,
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

In [ ]:
client = boto3.Session(profile_name=aws_profile_name).client("opensearchserverless",config=boto_config)

### Define encryption policy

In [ ]:
policy = """{
                "Rules": [
                    {
                        "Resource": [
                            "collection/sgrealestate-collection"
                        ],
                        "ResourceType": "collection"
                    }
                ],
                "AWSOwnedKey": true
            }"""

try:
    response = client.create_security_policy(
        description="Singapore RealEstate Sale Data security policy",
        name="sgrealestate-data-encrptpolicy",
        policy=policy,
        type="encryption",
    )
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)

### Define network policy

In [ ]:
policy = """[
                {
                    "Rules": [
                        {
                            "Resource": [
                                "collection/sgrealestate-collection"
                            ],
                            "ResourceType": "collection"
                        }
                    ],
                    "AllowFromPublic": true
                }
            ]"""

try:
    response = client.create_security_policy(
        description="Singapore RealEstate Network Policy",
        name="sgrealestate-nw-encrptpolicy",
        policy=policy,
        type="network",
    )
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)

### Define data access policy

In [ ]:
#Put your own  IAM principle ARN
policy = """[
                {
                    "Rules": [
                        {
                            "Resource": [
                                "collection/*"
                            ],
                            "Permission": [
                                "aoss:*"
                            ],
                            "ResourceType": "collection"
                        },
                        {
                            "Resource": [
                                "index/*/*"
                            ],
                            "Permission": [
                                "aoss:*"
                            ],
                            "ResourceType": "index"
                        }
                    ],
                    "Principal": [
                        "your_arn_principal" 
                    ],
                    "Description": "Rule 1"
                }
            ]"""

try:
    response = client.create_access_policy(
        description="Singapore RealEstate Data Access Policy",
        name="sgrealestate-data-accesspolicy",
        policy=policy,
        type="data"
    )
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)

### Create the collection

In [ ]:
# create vector search collection
try:
    response = client.create_collection(
        description="SG RealEstate Data collection",
        name="sgrealestate-collection",
        type="VECTORSEARCH"
    )
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex) 

### Launch OpenSearch Client to Create the elements 

In [ ]:
service = "aoss"
credentials = boto3.Session(profile_name=aws_profile_name).get_credentials()
auth = AWSV4SignerAuth(credentials, aws_region_name, service)
print(auth)

In [ ]:
client = OpenSearch(
    hosts = [{"host": opensearch_host_name, "port": 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

### Setup Index on OpenSearch Serverless

In [ ]:
#1536 is dimension type of AWS Titan Embedding Model
index_body = {
    "mappings": {
        "properties": {
            "sg_realestate_raw_data": {"type": "text"},
            "sg_realestate_data_vector": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {
                    "engine": "nmslib",
                    "space_type": "cosinesimil",
                    "name": "hnsw",
                    "parameters": {"ef_construction": 512, "m": 16},
                },
            },
        }
    },
    "settings": {
        "index": {
            "number_of_shards": 2,
            "knn.algo_param": {"ef_search": 512},
            "knn": True,
        }
    },
}

try:
    response = client.indices.create(index_name, body=index_body)
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)

### Verify the index

In [ ]:
try:
    response = client.indices.get(index_name)
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)